In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
import matplotlib.pyplot as plt


In [3]:
# Read in the data 
df = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv',index_col = 'Id')

In [4]:
## Exploratory Data Analysis
df.describe()

# The dataframe has 1460 rows and 80 columns, with SalePrice being the target variable (supervised learning).

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [5]:
# Create a function to find the most correlated features

def find_most_correlated_features(df, target_col, num_features=10):
      """Finds the most correlated features with the target variable."""
      corr_with_target = df.corr()[target_col].abs().sort_values(ascending=False)
      most_correlated = corr_with_target.drop(target_col).head(num_features)
      return most_correlated

In [6]:
def isolate_numerical_features(df):
  """
  Isolates numerical features from a pandas DataFrame.

  Args:
    df: The input pandas DataFrame.

  Returns:
    A pandas DataFrame containing only the numerical features.
  """
  numerical_features = df.select_dtypes(include=['number'])
  return numerical_features

In [7]:
df_numerical  = isolate_numerical_features(df)
df_numerical.shape

(1460, 37)

In [8]:
find_most_correlated_features(df_numerical,'SalePrice',num_features=20)

OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
GarageYrBlt     0.486362
MasVnrArea      0.477493
Fireplaces      0.466929
BsmtFinSF1      0.386420
LotFrontage     0.351799
WoodDeckSF      0.324413
2ndFlrSF        0.319334
OpenPorchSF     0.315856
HalfBath        0.284108
LotArea         0.263843
Name: SalePrice, dtype: float64

In [9]:
##### Building model #####

In [10]:
# Building a decision tree
from sklearn.tree import DecisionTreeRegressor

house_model = DecisionTreeRegressor(random_state =1)

In [11]:
X = df[['OverallQual','GrLivArea','GarageCars','GarageArea','TotalBsmtSF','1stFlrSF','FullBath','TotRmsAbvGrd','YearBuilt','YearRemodAdd','GarageYrBlt','MasVnrArea','Fireplaces','BsmtFinSF1','LotFrontage','WoodDeckSF','2ndFlrSF','OpenPorchSF','HalfBath','LotArea']]
y = df['SalePrice']

In [12]:
# Combine X and y for easier row-wise dropping
data = pd.concat([X, y], axis=1)

# Drop rows with any NaN values
data_cleaned = data.dropna()

In [13]:
data_cleaned.head()

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,MasVnrArea,Fireplaces,BsmtFinSF1,LotFrontage,WoodDeckSF,2ndFlrSF,OpenPorchSF,HalfBath,LotArea,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,7,1710,2,548,856,856,2,8,2003,2003,...,196.0,0,706,65.0,0,854,61,1,8450,208500
2,6,1262,2,460,1262,1262,2,6,1976,1976,...,0.0,1,978,80.0,298,0,0,0,9600,181500
3,7,1786,2,608,920,920,2,6,2001,2002,...,162.0,1,486,68.0,0,866,42,1,11250,223500
4,7,1717,3,642,756,961,1,7,1915,1970,...,0.0,1,216,60.0,0,756,35,0,9550,140000
5,8,2198,3,836,1145,1145,2,9,2000,2000,...,350.0,1,655,84.0,192,1053,84,1,14260,250000


In [14]:
X_cleaned = data_cleaned.iloc[:, :-1]  # Assuming the target is the last column
y_cleaned = data_cleaned.iloc[:, -1] # Target column is SalePrice.

In [15]:
# Split the cleaned data into train, test, cross-validation sets

#X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Get 60% of the dataset as the training set
X_train, X_, y_train, y_ = train_test_split(X_cleaned, y_cleaned, test_size = 0.4, random_state =42)
# Random state ensures that the split is the same every time you run the code, for reproducibility.

# Split the 40% subset into 20% for cross validation and 20% for test set 
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size =0.5, random_state=42)

# Delete temporary variables
del X_, y_

print(f"the shape of the training set (input) is: {X_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {X_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")
print(f"the shape of the test set (input) is: {X_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")

the shape of the training set (input) is: (672, 20)
the shape of the training set (target) is: (672,)

the shape of the cross validation set (input) is: (224, 20)
the shape of the cross validation set (target) is: (224,)

the shape of the test set (input) is: (225, 20)
the shape of the test set (target) is: (225,)


In [16]:
house_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1)

In [17]:
# Make predictions
predictions = house_model.predict(X_test)

In [18]:
# Evaluate the model: MAE, MSE, RMSE
MAE = mean_absolute_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(MSE)

print(MAE)
# With an MAE of 22,868, this means that on average my model's predictions are off by £22,688. Avg house price in dataset is £180,921. On average 12.5% error. 
print(MSE)
print(RMSE)
# RMSE of first iteration of the model is £33,189. RMSE penalises larger errors more heavily than MAE.

30277.933333333334
2504992869.3377776
50049.903789495715


In [19]:
# Improving the model through hyperparameter tuning
# Using GridSearchCV

from sklearn.model_selection import GridSearchCV

In [20]:
# Create a 2nd model that will use for hyperparameter training
# This is the model that will be tuned.

house_model_v2 = DecisionTreeRegressor(random_state=42)

In [21]:
# Parameters that we will vary with GridSearch.
# Parameter grid. It's a dictionary that specifies the hyperparemeters that will be tuned.

# Max_depth is the maximum depth of the tree.
# Min_samples_splt is the min number of samples required to split an internal node.
# Min_samples_leaf is the min number of samples required to be at a leaf node.
# Max_features is the number of features to consider when looking for the best split.

param_grid = {
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 3, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2', None]  
}

In [22]:
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
# Have to choose a scoring metric to evaluate the performance of each hyperparameter combination.

# Using negative mean squared error as the scoring metric
scoring_metric = 'neg_mean_squared_error'

# Alternatively, to minimize MAE:
# scoring_metric = 'neg_mean_absolute_error'

# Or to maximize R-squared:
# scoring_metric = 'r2'

In [23]:
grid_search = GridSearchCV(estimator=house_model_v2,
                           param_grid=param_grid,
                           scoring=scoring_metric,
                           cv=5, # The number of cross-validation folds. Common values is 5 or 10.
                           n_jobs=-1, # uses all CPU cores
                           verbose=2) # Controls the amount of output during the search.

In [24]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': ['auto', 'sqrt', 'log2', None],
                         'min_samples_leaf': [1, 3, 5, 10],
                         'min_samples_split': [2, 5, 10, 20]},
             scoring='neg_mean_squared_error', verbose=2)

In [25]:
print("Best Hyperparameters:", grid_search.best_params_)

Best Hyperparameters: {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 10}


In [26]:
print("Best Score:", grid_search.best_score_)

Best Score: -1770726204.4919262


In [27]:
best_model = grid_search.best_estimator_

In [28]:
predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Performance on Test Set (Best Model):")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Performance on Test Set (Best Model):
Mean Squared Error: 2157101966.741212
Mean Absolute Error: 28862.319156966492
R-squared: 0.7170942124981097


In [29]:
print(np.sqrt(mse))

46444.6118160246


In [30]:
# With GridSearch, I've gotten my MSE down to £46,444 from £50k and my MAE is now £28k down from £30k.